In [1]:
import importlib
import mcp_server
importlib.reload(mcp_server)
from mcp_server import server
import asyncio


In [2]:
tools = await server()


Available tools on the server:

• get_current_user_details — Fetch member's profile, miles, identity.Tip: Call before other tools (i.e list_user_flights)
• get_expiring_miles — Calculate member's expiring miles. Tip: Call after `get_current_user_details` to ensure member details are available.
• list_user_flights — Fetch member's flights. Input: surname. Should be called after `get_current_user_details`. Tip: Call if the PNR has not yet been provided.
• search_flights — Search THY (Turkish Airlines) outbound flights. Based on information from user origin(s), destination(s), date(s), and passenger details to find available flights and their prices.Important: For round trips, ALWAYS call search_inbound_flights after selecting an outbound to get accurate total pricing and return options.
• search_inbound_flights — Required step after selecting outbound flight. Provides return flight options with accurate combined pricing including all taxes and fees.Provide details of the chosen outbound

In [48]:
tools[3]

Tool(name='search_flights', title=None, description='Search THY (Turkish Airlines) outbound flights. Based on information from user origin(s), destination(s), date(s), and passenger details to find available flights and their prices.Important: For round trips, ALWAYS call search_inbound_flights after selecting an outbound to get accurate total pricing and return options.', inputSchema={'type': 'object', 'properties': {'originDestinations': {'type': 'array', 'items': {'type': 'object', 'properties': {'departureDateTime': {'type': 'object', 'properties': {'departureDate': {'type': 'string', 'pattern': '^\\d{2}-\\d{2}-\\d{4} \\d{2}:\\d{2}$'}}, 'required': ['departureDate'], 'additionalProperties': False, 'description': 'Details of the departure date and time for this segment.'}, 'originAirportCode': {'type': 'string', 'minLength': 3, 'maxLength': 3, 'description': "The 3-letter IATA code for the origin airport. Example: 'IST' for Istanbul Airport."}, 'originCountryCode': {'type': 'string'

In [52]:
tools[3].inputSchema["properties"]["passengers"]

{'type': 'array',
 'items': {'type': 'object',
  'properties': {'passengerType': {'type': 'string',
    'enum': ['ADT', 'CHD', 'INF', 'YAD', 'SRC', 'STD', 'CNS', 'STU'],
    'description': "Passenger type code. Common types: 'ADT' (Adult), 'CHD' (Child), 'INF' (Infant). Ensure to use a valid type from the allowed list."},
   'quantity': {'type': 'integer',
    'exclusiveMinimum': 0,
    'description': 'Number of passengers of this specific type. Must be a positive integer. Example: 1'}},
  'required': ['passengerType', 'quantity'],
  'additionalProperties': False,
  'description': 'Specifies the type and quantity of passengers.'},
 'minItems': 1,
 'description': 'An array specifying the number of passengers for each type (e.g., adults, children, infants).'}

In [32]:
tools[3].inputSchema

{'type': 'object',
 'properties': {'originDestinations': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'departureDateTime': {'type': 'object',
      'properties': {'departureDate': {'type': 'string',
        'pattern': '^\\d{2}-\\d{2}-\\d{4} \\d{2}:\\d{2}$'}},
      'required': ['departureDate'],
      'additionalProperties': False,
      'description': 'Details of the departure date and time for this segment.'},
     'originAirportCode': {'type': 'string',
      'minLength': 3,
      'maxLength': 3,
      'description': "The 3-letter IATA code for the origin airport. Example: 'IST' for Istanbul Airport."},
     'originCountryCode': {'type': 'string',
      'minLength': 2,
      'maxLength': 2,
      'description': "The 2-letter ISO country code for the origin airport's country. Example: 'TR' for Turkey."},
     'originMultiport': {'type': 'boolean',
      'default': True,
      'description': 'Optional. Set to true if the origin is a city with multiple airports a

In [ ]:
input_schema = tools[3].inputSchema

In [ ]:
# her fonksiyonun inputSchema'sı var, bu inputSchema'da properties var
# bu propoerties fonksiyonun istediği parametereler
# her parametrenin enum kısmı kabul ettiği classları belirtiyor.
# her property bir description içeriyor.

In [43]:
args_desc = []
if "properties" in input_schema:
    for param_name, param_info in input_schema["properties"].items():
        arg_desc = (
            f"- {param_name}: {param_info.get('description', 'No description')}"
        )

        args_desc.append(arg_desc)


In [44]:
args_desc

["- originDestinations: An array defining the flight segments. For a one-way trip, this will have one item. For a round trip, it will has two items (outbound and return). For multi-city, it can have more.- For a 'round' trip, this MUST be an array with exactly TWO items (outbound and return).",
 '- passengers: An array specifying the number of passengers for each type (e.g., adults, children, infants).',
 "- tripType: Specifies the trip type. This choice STRICTLY determines the number of flight segments required in the 'originDestinations' array.\n      - 'one_way': REQUIRES exactly ONE flight segment.\n      - 'round': REQUIRES exactly TWO flight segments (outbound and return).\n      - 'multi': REQUIRES two or more flight segments. 'round' indicates you are starting a flow for `search_inbound_flights` tool",
 '- fareFamily: No description']